# Unita: Convolutional Neural Networks

### Architetture basate su CNN 

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
# Python 2 and 3 Compatibility Library
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = "DeepLearning/data/notMNIST.pickle"

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  

image_size = 28
num_labels = 10
# La profondita' dei dati in input
num_channels = 1 # grayscale 

def reformat(dataset, labels):
  # Questa volta il dataset lo organizziamo in matrici 
  # di dimensioni <image_size,image_size,num_channels>
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## PROBLEMA #1

Costruiamo una Convnet con 2 convolutional layers e un full-connected layer.

Il numero di parametri aumenta, attenzione al numero di nodi da definire.

### tf.nn.conv2d()

Prende in input un tensore [filter_height, filter_width, in_channels, out_channels]

In [4]:
# Quanto e' la dimensione dei due convolutional layers?

output_1 = (28.00 - 5.00 - (-2.00)) / 2.00 + 1.00
print(np.ceil(output_1))
output_2 = (output_1 - 5.00 - (-2.00)) / 2.00 + 1.00
print(np.ceil(output_2))

14.0
7.0


In [4]:
# Definiamo una funzione che, data la dimensione del layer in input, 
# la dimensione del filtro, padding (SAME o VALID), restituisca la 
# dimensione del layer ottenuto applicando il filtro al layer in input 2 volte.
def output_size_no_pool(input_size, filter_size, padding='SAME', stride=2):
    if padding == 'SAME':
        padding = -1.00
    elif padding == 'VALID':
        padding = 0.00
    else:
        return None
    output_1 = float(((input_size - filter_size - 2*padding) / stride) + 1.00)
    output_2 = float(((output_1 - filter_size - 2*padding) / stride) + 1.00)
    return int(np.ceil(output_2))

patch_size = 5
final_image_size = output_size_no_pool(image_size, patch_size, padding='SAME', stride=2)
print(final_image_size)

7


In [6]:
batch_size = 16
# Dimensione del local receptive field 5x5
patch_size = 5
# Numero di feature map (o profondita' del filtro)
depth = 16
# Nodi dell'hidden layer
num_hidden = 64
# Stride -> Numero di celle che il filtro scorre ad ogni iterazione
stride = 2

graph = tf.Graph()

with graph.as_default():

  # Abbiamo in input una struttura composta da una minibatch di matrici 
  # di dimensioni: image_size x image_size x num_channels 
  # (nel nostro caso 28x28x1)
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))  
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Generiamo i valori iniziali per ogni filtro.

  # La definizione dei pesi include anche la struttura del nostro filtro.
  # Ogni filtro e' un tensore di 4 dimensioni: 
  # [filter_height, filter_width, in_channels, out_channels]
  # filter_height e filter_width definiscono la dimensione del Local receptive field.
  # in_channels e' la profondita' del tensore in input 
  # (nel nostro caso 1)
  # out_channels e' la profondita' del convolution layer 
  # (nel nostro caso 16).

  # Input depth: 1
  # Output depth: 16
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  # layer 2
  # Input depth: 16
  # Output depth: 16    
  # Il filtro ha la stessa dimensione del precedente, 
  # ma l'input depth corrisponde all'output depth del layer precedente.
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  # layer 3 
  # E' un hidden layer (full-connected) che raccoglie l'output del layer precedente.
  # Abbiamo applicato 2 conv layer con stride = 2 
  # Con stride 2 la dimensione dell'input si dimezza ogni volta.
  # Per questo abbiamo un input di dimensioni: 
  # ((image_size / 4) * (image_size / 4) * depth)
  # Il layer ha 64 nodi.
  final_image_size = output_size_no_pool(image_size, patch_size, padding='SAME', stride=stride)
  layer3_weights = tf.Variable(tf.truncated_normal(
      [final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  # layer di output
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Per chiarezza definisco una funzione model che raggruppa tutte le operazioni 
  def model(data):        
    # tf.nn.conv2d prende in input: i dati su cui operare la convoluzione, 
    # il tensore che rappresenta il filtro,
    # un vettore di 4 int corrispondenti agli stride.
    # Di questo ultimo vettore, ci interessano i due valori centrali 
    # (stride di 2 celle in verticale e orizzontale).
    # Aggiungo 0-padding all'input (padding=SAME) 
    # in modo che la finestra del filtro (5x5) scorra su tutti i dati (28x28).
    conv = tf.nn.conv2d(data, layer1_weights, [1, stride, stride, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, stride, stride, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    
    # Leggiamo le dimensioni del tensore a valle del secondo layer + RELU
    shape = hidden.get_shape().as_list()
    # Adatto il tensore ottenuto dalla 2a convoluzione 
    # in modo da poterlo moltiplicare con il 3o layer 
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  logits = model(tf_train_dataset)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))    
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
# Ridurre se impiega troppo tempo
num_steps = 3000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.948508
Minibatch accuracy: 6.2%


Validation accuracy: 10.0%


Minibatch loss at step 500: 0.811308
Minibatch accuracy: 81.2%


Validation accuracy: 80.8%


Minibatch loss at step 1000: 0.501610
Minibatch accuracy: 81.2%


...

Minibatch loss at step 2000: 0.309435
Minibatch accuracy: 87.5%


Validation accuracy: 84.5%


Minibatch loss at step 2500: 0.728022
Minibatch accuracy: 75.0%


Validation accuracy: 85.2%


Test accuracy: 92.6%


## PROBLEMA #2

Alterare la rete considerando un max pooling con stride 2 e dimensioni del filtro pari a 2.

### nn.max_pool()

In [8]:
# Stessi iperparametri
batch_size = 16
depth = 16
num_hidden = 64
stride = 2
pooling_size = 2

graph = tf.Graph()

with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, 
                                             image_size, 
                                             image_size, 
                                             num_channels))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, 
                                            num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, 
                                                      patch_size, 
                                                      num_channels, 
                                                      depth], 
                                                     stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, 
                                                      patch_size, 
                                                      depth, depth], 
                                                     stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # Sebbene la convoluzione ora ha stride 1, ho un pooling con stride 2
    # Percio' la dimensione del layer 3 e' come il caso precedente.
    layer3_weights = tf.Variable(tf.truncated_normal([final_image_size * 
                                                      final_image_size * 
                                                      depth, num_hidden],
                                                    stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], 
                                                     stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    
    def model(data):
        # La convolution la faccio con stride = 1, il pooling lo applico dopo
        conv_1 = tf.nn.conv2d(data, 
                              layer1_weights, 
                              strides=[1, 1, 1, 1], 
                              padding='SAME')
        hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
        
        # La funzione max_pool prende come parametro: 
        # il layer su cui calcolare il max pooling, 
        # un tensore di 4 dimensioni con la dimensione del pool 
        # (ci interessano i due valori centrali),
        # le dimensioni dello stride, e il padding.
        pool_1 = tf.nn.max_pool(hidden_1, 
                                [1, pooling_size, pooling_size, 1], 
                                [1, stride, stride, 1], 
                                padding='SAME')
        
        conv_2 = tf.nn.conv2d(pool_1, 
                              layer2_weights, 
                              strides=[1, 1, 1, 1], 
                              padding='SAME')
        hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
        pool_2 = tf.nn.max_pool(hidden_2, 
                                [1, pooling_size, pooling_size, 1], 
                                [1, stride, stride, 1], 
                                padding='SAME')
        
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, 
                             [shape[0], 
                              shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, 
                                                                  logits=logits))    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 30000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, 
                     tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, 
                                         loss, 
                                         train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 5000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.448927
Minibatch accuracy: 6.2%


Validation accuracy: 10.0%


Minibatch loss at step 5000: 0.316276
Minibatch accuracy: 87.5%


Validation accuracy: 87.7%


Minibatch loss at step 10000: 0.321879
Minibatch accuracy: 93.8%


...

Minibatch loss at step 20000: 0.365715
Minibatch accuracy: 87.5%


Validation accuracy: 90.2%


Minibatch loss at step 25000: 0.236400
Minibatch accuracy: 93.8%


Validation accuracy: 90.5%


Test accuracy: 94.9%


## PROBLEMA #3

Prova ad aumentare l'efficienza aggiungendo Dropout e usando il Learning rate decay.

### Proviamo con la configurazione: 

Convolution -> Pooling -> Convolution -> Pooling -> Fully-connected -> Fully-connected -> Readout

Pooling = Average Pooling con dimensione 2 e pool_stride = 2; padding='VALID'

Per prevenire overfitting applico: Learning rate decay + L2 Regularization + Dropout

In [15]:
image_size = 28
# Consideriamo anche il pooling
def output_size_pool(input_size, conv_filter_size, pool_filter_size, padding, conv_stride, pool_stride):
    if padding == 'SAME':
        padding = -1.00
    elif padding == 'VALID':
        padding = 0.00
    else:
        return None
    # Dopo la prima convoluzione
    output_1 = (((input_size - conv_filter_size - 2*padding) / conv_stride) + 1.00)
    # Dopo il primo pooling
    # Uso la stessa formula poiche' il pooling ha lo stride pari a 2, 
    # percio' riduce l'input come la convoluzione
    output_2 = (((output_1 - pool_filter_size - 2*padding) / pool_stride) + 1.00)    
    # Dopo la seconda convoluzione
    output_3 = (((output_2 - conv_filter_size - 2*padding) / conv_stride) + 1.00)
    # Dopo il secondo pooling
    output_4 = (((output_3 - pool_filter_size - 2*padding) / pool_stride) + 1.00)  
    return int(output_4)

final_image_size = output_size_pool(input_size=image_size, 
                                    conv_filter_size=5, 
                                    pool_filter_size=2, 
                                    padding='VALID', 
                                    conv_stride=1, 
                                    pool_stride=2)
print(final_image_size)

In [16]:
batch_size = 16
depth = 32
num_hidden = 64
beta = 0.001
conv_filter_size = 5
pool_filter_size = 2 
conv_stride = 1
pool_stride = 2
# Dropout prob
keep_prob = 0.5


graph = tf.Graph()

with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, 
                                             image_size, 
                                             image_size, 
                                             num_channels))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, 
                                            num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, 
                                                      patch_size, 
                                                      num_channels, 
                                                      depth], 
                                                     stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, 
                                                      patch_size, 
                                                      depth, 
                                                      depth], 
                                                     stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    # Ricalcolo la dimensione del layer in base alla nuova configurazione 
    final_image_size = output_size_pool(input_size=image_size, 
                                        conv_filter_size=conv_filter_size,
                                        pool_filter_size=pool_filter_size, 
                                        padding='VALID',
                                        conv_stride=conv_stride, 
                                        pool_stride=pool_stride)
    layer3_weights = tf.Variable(tf.truncated_normal([final_image_size * 
                                                      final_image_size * 
                                                      depth, num_hidden],
                                                     stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, 
                                            shape=[num_hidden]))    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, 
                                                      num_hidden], 
                                                     stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, 
                                            shape=[num_hidden]))
    
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, 
                                                      num_labels], 
                                                     stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, 
                                            shape=[num_labels]))

    def model(data):
        # Convolutional layer #1 con Pooling
        conv_1 = tf.nn.conv2d(data, layer1_weights, 
                              strides=[1, conv_stride, conv_stride, 1], 
                              padding='VALID')
        hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
        # Il metodo tf.nn.avg_pool calcola l'average pooling
        pool_1 = tf.nn.avg_pool(hidden_1, 
                                [1, pool_filter_size, pool_filter_size, 1], 
                                [1, pool_stride, pool_stride, 1], 
                                padding='VALID')
        
        # Convolutional layer #2 con Pooling
        conv_2 = tf.nn.conv2d(pool_1, layer2_weights, 
                              strides=[1, conv_stride, conv_stride, 1], 
                              padding='VALID')
        hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
        pool_2 = tf.nn.avg_pool(hidden_2, 
                                [1, pool_filter_size, pool_filter_size, 1], 
                                [1, pool_stride, pool_stride, 1], 
                                padding='VALID')
        
        # Full-connected layer #1 con Dropout
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, 
                             [shape[0], 
                              shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, 
                                      layer3_weights) + layer3_biases)
        hidden_drop = tf.nn.dropout(hidden, 
                                    keep_prob)
        
        # Full-connected layer #2 con Dropout
        hidden_2 = tf.nn.relu(tf.matmul(hidden_drop, 
                                        layer4_weights) + layer4_biases)
        hidden_2_drop = tf.nn.dropout(hidden_2, 
                                      keep_prob)
        
        return tf.matmul(hidden_2_drop, layer5_weights) + layer5_biases

    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, 
                                                                  logits=logits))    
    
    # Loss function con L2 Regularization 
    # N.B. La regolarizzazione si applica solitamente alle reti full-connected.
    regularizers = tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer5_weights)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Decaying learning rate
    global_step = tf.Variable(0)  
    start_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(start_learning_rate, 
                                               global_step, 
                                               100000, 
                                               0.96, 
                                               staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [17]:
num_steps = 30000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 5000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.456016
Minibatch accuracy: 12.5%


Validation accuracy: 10.4%


Minibatch loss at step 5000: 0.525465
Minibatch accuracy: 75.0%


Validation accuracy: 81.4%


Minibatch loss at step 10000: 0.863676
Minibatch accuracy: 75.0%


...

Minibatch loss at step 20000: 0.519225
Minibatch accuracy: 87.5%


Validation accuracy: 86.2%


Minibatch loss at step 25000: 0.465125
Minibatch accuracy: 87.5%


Validation accuracy: 86.1%


Test accuracy: 92.6%
